# Electricity Usage Analysis

In [1]:
for name in dir():
    if not name.startswith("_"):
        del globals()[name]

In [2]:
import sys
!{sys.executable} -m pip install hvplot

In [27]:
# Libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import hvplot.pandas

from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [5]:
df = pd.read_csv('df3.csv', header='infer')
df.set_index('date', inplace=True)
df['Hour'] = df['Hour'].astype(str)

In [6]:
print(df.shape)
df.head()

(16459, 29)


,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,v1,Weekday,Hour
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.20,48.900000,17.033333,45.53,6.600000,733.5,92.000000,7.000000,63.000000,5.3,13.275433,Mon,17
2016-01-11 17:10:00,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,21.79,48.863333,18.890000,45.56,6.483333,733.6,92.000000,6.666667,40.000000,5.2,18.606195,Mon,17
2016-01-11 17:20:00,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.20,48.730000,17.000000,45.50,6.366667,755.0,92.000000,6.333333,55.333333,5.1,28.642668,Mon,17
2016-01-11 17:30:00,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.10,48.590000,17.000000,45.40,6.250000,733.8,84.333333,6.000000,51.500000,5.0,45.410390,Mon,17
2016-01-11 17:40:00,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,34.927778,18.10,48.590000,17.000000,45.40,6.133333,733.9,84.333333,5.666667,47.666667,4.9,10.084097,Mon,17


In [7]:
df.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,v1
count,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.00000,16459.000000,16459.000000
mean,96.233064,3.955283,21.411526,40.169764,20.036250,40.440233,21.935220,39.375334,20.468408,39.095497,19.286999,51.174386,7.166662,59.619036,19.787542,35.457365,21.692460,43.134914,19.029985,41.566813,6.848526,754.794260,80.719657,4.120142,38.61759,3.335968,24.934725
std,102.028297,8.060528,1.389577,3.513932,1.875805,3.643092,1.668433,3.038812,1.807956,3.971781,1.548120,8.694324,5.296161,28.978911,1.760854,4.457492,1.717435,4.855274,1.674996,3.856741,4.901699,6.937394,14.110826,2.358126,10.59493,3.680742,14.517159
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.596667,17.200000,30.663333,15.100000,27.660000,15.330000,29.815000,-6.030000,1.000000,15.390000,23.200000,16.362222,29.600000,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.00000,-6.600000,0.005322
25%,50.000000,0.000000,20.790000,37.900000,18.890000,38.400000,20.823333,37.290000,19.390000,36.290000,18.200000,46.090000,3.933333,40.725000,18.700000,32.500000,20.790000,39.568036,18.000000,38.976500,3.663889,751.100000,73.000000,2.333333,32.50000,1.166667,12.425313
50%,60.000000,0.000000,21.390000,39.626667,19.760000,40.590000,21.890000,38.730000,20.390000,38.500000,19.260000,49.400000,6.656667,62.261667,19.600000,34.927778,21.790000,42.600000,18.890000,41.000000,6.400000,755.000000,84.333333,3.833333,40.00000,3.200000,24.774977
75%,100.000000,0.000000,22.100000,42.433333,20.790000,42.826667,22.790000,41.290000,21.390000,41.465000,20.200000,53.334000,9.726667,84.945000,20.926667,37.905409,22.730000,46.060000,20.251429,43.760000,9.600000,759.266667,91.166667,5.333333,40.00000,5.486111,37.583769
max,1080.000000,70.000000,25.700000,63.360000,29.856667,56.026667,27.600000,50.163333,26.200000,51.090000,25.745000,96.321667,28.236000,99.900000,25.390000,51.400000,27.230000,58.780000,24.200000,53.326667,26.100000,772.300000,100.000000,14.000000,66.00000,15.500000,49.996530


In [203]:
# some testing ...
if 0:
    x = df3['Appliances'] - df3['Appliances'].min() + 1
    print(x[0:10].tolist())

    xbox, opt_lambda = boxcox(x)
    print(xbox[0:10].tolist())
    print(opt_lambda)

    xinvbox = inv_boxcox(xbox, opt_lambda)
    print(xinvbox[0:10].tolist())

[51.0, 51.0, 41.0, 41.0, 51.0, 41.0, 51.0, 51.0, 51.0, 61.0]
[2.646922942509263, 2.646922942509263, 2.5515385251450624, 2.5515385251450624, 2.646922942509263, 2.5515385251450624, 2.646922942509263, 2.646922942509263, 2.646922942509263, 2.7218760785681684]
-0.21655783519823987
[51.00000000000002, 51.00000000000002, 41.00000000000001, 41.00000000000001, 51.00000000000002, 41.00000000000001, 51.00000000000002, 51.00000000000002, 51.00000000000002, 60.99999999999995]


In [8]:
# df
def get_filtered_df(df, excluded_cols):
    features = [col for col in df.columns if col not in excluded_cols]
    X = df[features].copy()
    y = df['Appliances']
    return (X, y)

In [9]:
def log_transform(v, subtract_min=1, add_one=0):
    vt = v
    if (subtract_min):
        vt = v - v.min() + 1
    elif (add_one):
        vt = v + 1
    
    return np.log(vt)

In [10]:
# log transformation of selected features in a dataaframe
def log_transform_features(X, features, subtract_min=1, add_one=0):
    Xt = X.copy()
    for col in features:
        vt = log_transform(X[col], subtract_min, add_one)
        Xt[col] = vt

    return (Xt)

In [11]:
def boxcox_transform(v, subtract_min=1):
    v1 = v
    vmin = v.min()
    if (subtract_min):
        v1 = v - vmin + 1
    vt, opt_lambda = boxcox(v1)
    return (vt, vmin, opt_lambda)

In [12]:
# Box-Cox transformation of selected features in a dataaframe
def boxcox_transform_features(X, features):
    Xt = X.copy()
    opt_lambda_dict = {}
    for col in features:
        vt, vmin, opt_lambda = boxcox_transform(X[col])
        Xt[col] = vt
        opt_lambda_dict[col] = opt_lambda

    return (Xt, opt_lambda_dict)

In [15]:
def one_hot_encoding(X, cat_col_list):
    df_cat = X[cat_col_list]

    encoding_cat = OneHotEncoder()
    encoding_cat.fit(df_cat)
    df_cat_enc = pd.DataFrame(
        encoding_cat.transform(df_cat).toarray(), columns=encoding_cat.get_feature_names()
    )

    df_cat_enc.set_index(df_cat.index, inplace=True)
    Xt = pd.merge(X, df_cat_enc, how='left', left_index=True, right_index=True)
    Xt = Xt.drop(cat_col_list, axis=1)
    
    return (Xt)

In [13]:
#scoring = make_scorer(r2_score)
scoring = make_scorer(root_mean_squared_error, greater_is_better=False)

In [14]:
#-------------------------------------------------------------------------------
# Execute the cross-validation for all parameter combinations and plots the
# evaluation metrics. At each iteration of the cross validation process, the 
# model is evaluated using all the below metrics but the optimal model is 
# determined using the AUC score (as this minimizes both false positives &
# false negatives).
#-------------------------------------------------------------------------------
def build_optimal_model(model_type, model, model_parameter_grid, X, y, scoring):
    grid_object = GridSearchCV(model, param_grid=model_parameter_grid, cv=5, scoring=scoring, refit=True, return_train_score=True, n_jobs=-1)
    print('Performing grid search to determine the optimal parameters ...')
    grid_object.fit(X, y)
    grid_search_output = pd.DataFrame(grid_object.cv_results_)

    optimal_model = grid_object.best_estimator_
    optimal_model = optimal_model.fit(X, y)

    return grid_object, grid_search_output, optimal_model

In [16]:
# Ridge regression setup
model_type = 'Linear Model'
model = Ridge(copy_X=False)
model_parameter_grid = {
         'normalize': [True, False],
         'alpha': [0, 0.001, 0.01, 0.05, 0.1, 0.5, 1]
        }

In [17]:
%%time

excluded_cols = ['Appliances', 'Weekday', 'Hour']
X, y = get_filtered_df(df, excluded_cols)

# log transform of Appliances variable
Xt = X
yt = log_transform(y, subtract_min=0)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

Performing grid search to determine the optimal parameters ...
CPU times: user 1.3 s, sys: 273 ms, total: 1.58 s
Wall time: 2.63 s


In [19]:
print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'coef': optimal_model.coef_}).sort_values('coef', ascending=False)

0.5977335089944273
{'alpha': 0.05, 'normalize': True}


,feature,coef
0,lights,20.119781
2,RH_1,19.962973
5,T3,19.053394
15,T8,11.209785
6,RH_3,8.797314
11,T6,5.553496
10,RH_5,3.489372
3,T2,2.781603
19,T_out,1.866760
22,Windspeed,1.752574


In [20]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(y, np.exp(predicted)))

0.5812281481041298
98.70622856552946
0.22159871512311147
0.06400348911715137


In [22]:
%%time

# 2. Box-Cox transformation of features

excluded_cols = ['Appliances', 'Weekday', 'Hour']
X, y = get_filtered_df(df, excluded_cols)

Xt, opt_lambda_dict = boxcox_transform_features(X, X.columns.tolist())
yt, _, opt_lambda = boxcox_transform(y, subtract_min=0)
opt_lambda_dict['Appliances'] = opt_lambda # Appliances > 0

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

Performing grid search to determine the optimal parameters ...
CPU times: user 1.5 s, sys: 37.4 ms, total: 1.54 s
Wall time: 477 ms


In [23]:
print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'coef': optimal_model.coef_}).sort_values('coef', ascending=False)

0.05658156256379262
{'alpha': 0.05, 'normalize': True}


,feature,coef
0,lights,2.031315
2,RH_1,1.746945
5,T3,1.510629
15,T8,1.174090
3,T2,0.590084
11,T6,0.526776
6,RH_3,0.511830
19,T_out,0.440486
10,RH_5,0.396229
12,RH_6,0.291279


In [24]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(inv_boxcox(predicted, opt_lambda_dict['Appliances']), y))

print(r2_score(yt, predicted))
print(r2_score(y, inv_boxcox(predicted, opt_lambda_dict['Appliances'])))

0.05536059542875124
101.00397373013176
0.24311669532665703
0.019918856548492236


In [28]:
%%time

# 3. One-hot encoding of Weekday & Hour features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

Xt = one_hot_encoding(X, ['Weekday', 'Hour'])
yt = log_transform(y, subtract_min=0, add_one=0)

#print(Xt.shape)
#Xt.head()

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'coef': optimal_model.coef_}).sort_values('coef', ascending=False)

Performing grid search to determine the optimal parameters ...
0.5538258696531965
{'alpha': 0.01, 'normalize': True}
CPU times: user 1.69 s, sys: 219 ms, total: 1.91 s
Wall time: 1.83 s


,feature,coef
5,T3,22.335213
0,lights,15.368073
43,x1_18,14.745988
15,T8,14.704659
6,RH_3,13.922963
42,x1_17,10.483675
44,x1_19,8.973289
36,x1_11,7.173118
2,RH_1,6.877059
37,x1_12,6.644773


In [29]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.5322354982566241
93.84781790138422
0.3472934942439606
0.1538770572000654


In [30]:
%%time

# 4. One-hot encoding of Weekday & Hour features followed by Box-Cox transformation of numerical features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

cat_col_list = ['Weekday', 'Hour']
features_to_transform = [col for col in X.columns if col not in cat_col_list]
Xt1, opt_lambda_dict = boxcox_transform_features(X, features_to_transform)
yt, _, opt_lambda = boxcox_transform(y, subtract_min=0)
opt_lambda_dict['Appliances'] = opt_lambda # Appliances > 0

Xt = one_hot_encoding(Xt1, cat_col_list)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'coef': optimal_model.coef_}).sort_values('coef', ascending=False)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.0529453126842919
{'alpha': 0.05, 'normalize': True}
CPU times: user 2.3 s, sys: 135 ms, total: 2.43 s
Wall time: 984 ms


,feature,coef
5,T3,1.560526
0,lights,1.493836
43,x1_18,1.270487
15,T8,1.235440
42,x1_17,0.898427
44,x1_19,0.896893
6,RH_3,0.725022
46,x1_20,0.656011
37,x1_12,0.595248
36,x1_11,0.554005


In [31]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(inv_boxcox(predicted, opt_lambda_dict['Appliances']), y))

print(r2_score(yt, predicted))
print(r2_score(y, inv_boxcox(predicted, opt_lambda_dict['Appliances'])))

0.05082725317096058
97.18192825821772
0.3619999677739726
0.0926890819724474


In [33]:
%%time

# 5. One-hot encoding of Weekday & Hour features followed by log transformation of numerical features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

cat_col_list = ['Weekday', 'Hour']
features_to_transform = [col for col in X.columns if col not in cat_col_list]
Xt1 = log_transform_features(X, features_to_transform, subtract_min=1, add_one=0)
yt = log_transform(y, subtract_min=0, add_one=0)

Xt = one_hot_encoding(Xt1, cat_col_list)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'coef': optimal_model.coef_}).sort_values('coef', ascending=False)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.5526373643530169
{'alpha': 0.05, 'normalize': True}
CPU times: user 1.81 s, sys: 225 ms, total: 2.04 s
Wall time: 2.05 s


,feature,coef
5,T3,15.578716
0,lights,15.430128
43,x1_18,14.373386
15,T8,12.861226
42,x1_17,9.996765
6,RH_3,9.647591
44,x1_19,8.794239
2,RH_1,6.337391
36,x1_11,6.055101
37,x1_12,5.887110


In [34]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.5341515322762614
94.46136266431134
0.3425855822314995
0.14277757139917557


# Experiments on scaled data

In [35]:
df = pd.read_csv('df3_scaled_range.csv', header='infer')
df.set_index('date', inplace=True)
df['Hour'] = df['Hour'].astype(str)

print(df.shape)
df.head()

(16459, 29)


,Appliances,Weekday,Hour,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,v1
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60.0,Mon,17,0.428571,0.347924,0.566187,0.225345,0.682849,0.249038,0.721368,0.351351,0.764262,0.176348,0.381691,0.381039,0.841827,0.181000,0.653428,0.169103,0.661412,0.230218,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449
2016-01-11 17:10:00,60.0,Mon,17,0.428571,0.347924,0.541326,0.225345,0.680944,0.249038,0.724444,0.351351,0.782437,0.176348,0.381691,0.375396,0.839872,0.181000,0.651064,0.499438,0.660155,0.429646,0.678532,0.369239,0.100000,0.894737,0.476190,0.600000,0.533937,0.372083
2016-01-11 17:20:00,50.0,Mon,17,0.428571,0.347924,0.530502,0.225345,0.678239,0.249038,0.731795,0.344745,0.778062,0.176348,0.380037,0.367420,0.830704,0.181000,0.646572,0.169103,0.655586,0.226638,0.676049,0.365488,0.597674,0.894737,0.452381,0.835897,0.529412,0.572848
2016-01-11 17:30:00,50.0,Mon,17,0.571429,0.347924,0.524080,0.225345,0.677204,0.249038,0.735214,0.341441,0.770949,0.176348,0.380037,0.363723,0.833401,0.174333,0.641489,0.159902,0.650788,0.226638,0.671909,0.361736,0.104651,0.793860,0.428571,0.776923,0.524887,0.908261
2016-01-11 17:40:00,60.0,Mon,17,0.571429,0.347924,0.531419,0.225345,0.675510,0.249038,0.735214,0.341441,0.762697,0.179549,0.380037,0.361777,0.848264,0.181000,0.415879,0.159902,0.650788,0.226638,0.671909,0.357985,0.106977,0.793860,0.404762,0.717949,0.520362,0.201611


In [36]:
%%time

# 3. One-hot encoding of Weekday & Hour features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

Xt = one_hot_encoding(X, ['Weekday', 'Hour'])
yt = log_transform(y, subtract_min=0, add_one=0)

#print(Xt.shape)
#Xt.head()

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'coef': optimal_model.coef_}).sort_values('coef', ascending=False)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.5538258696531964
{'alpha': 0.01, 'normalize': True}
CPU times: user 1.74 s, sys: 129 ms, total: 1.87 s
Wall time: 1.37 s


,feature,coef
5,T3,22.335213
0,lights,15.368073
43,x1_18,14.745988
15,T8,14.704659
6,RH_3,13.922963
42,x1_17,10.483675
44,x1_19,8.973289
36,x1_11,7.173118
2,RH_1,6.877059
37,x1_12,6.644773


In [37]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.532235498256624
93.84781790138419
0.3472934942439607
0.15387705720006617


In [38]:
%%time

# 4. One-hot encoding of Weekday & Hour features followed by Box-Cox transformation of numerical features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

cat_col_list = ['Weekday', 'Hour']
features_to_transform = [col for col in X.columns if col not in cat_col_list]
Xt1, opt_lambda_dict = boxcox_transform_features(X, features_to_transform)
yt, _, opt_lambda = boxcox_transform(y, subtract_min=0)
opt_lambda_dict['Appliances'] = opt_lambda # Appliances > 0

Xt = one_hot_encoding(Xt1, cat_col_list)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'coef': optimal_model.coef_}).sort_values('coef', ascending=False)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.052946184406459616
{'alpha': 0.05, 'normalize': True}
CPU times: user 2.12 s, sys: 152 ms, total: 2.27 s
Wall time: 1.51 s


,feature,coef
5,T3,1.537725
0,lights,1.520836
43,x1_18,1.266457
15,T8,1.217510
44,x1_19,0.900435
42,x1_17,0.896007
6,RH_3,0.710559
46,x1_20,0.663535
37,x1_12,0.592587
36,x1_11,0.551877


In [39]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(inv_boxcox(predicted, opt_lambda_dict['Appliances']), y))

print(r2_score(yt, predicted))
print(r2_score(y, inv_boxcox(predicted, opt_lambda_dict['Appliances'])))

0.050839924613537986
97.1860418943107
0.3616818160737354
0.09261226880356832


In [40]:
%%time

# 5. One-hot encoding of Weekday & Hour features followed by log transformation of numerical features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

cat_col_list = ['Weekday', 'Hour']
features_to_transform = [col for col in X.columns if col not in cat_col_list]
Xt1 = log_transform_features(X, features_to_transform, subtract_min=1, add_one=0)
yt = log_transform(y, subtract_min=0, add_one=0)

Xt = one_hot_encoding(Xt1, cat_col_list)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'coef': optimal_model.coef_}).sort_values('coef', ascending=False)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.552474106580279
{'alpha': 1, 'normalize': False}
CPU times: user 1.79 s, sys: 246 ms, total: 2.04 s
Wall time: 2.79 s


,feature,coef
5,T3,1.583365
0,lights,1.212695
15,T8,1.091359
6,RH_3,1.019509
2,RH_1,0.728160
43,x1_18,0.584864
42,x1_17,0.416920
44,x1_19,0.358441
10,RH_5,0.326735
1,T1,0.309107


In [41]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.5322166442761317
93.7788677867745
0.34733973654760375
0.15511989574114837
